In [1]:
import pandas as pd
import numpy as np
import copy
import warnings
import plotly
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn import metrics
from sklearn.model_selection import train_test_split
from PIL import Image 
import random
warnings.filterwarnings("ignore")
from tqdm import tqdm
import string

import torch
from torch import nn
from transformers import AutoTokenizer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)


In [3]:
news = pd.read_csv('/kaggle/input/articles/Articles.csv',sep=';')

In [4]:
news

,Web,Title,Url,Datetime,Abstract
0,idnes.cz,PŘEHLEDNĚ: Co nového nás čeká v roce 2023. Za co více zaplatíme i dostaneme,https://www.idnes.cz/zpravy/domaci/cesko-zmeny-v-roce-2023-prehledne.A221230_105559_domaci_knn,2023-01-01,"Česko si zvolí nového prezidenta, na pražském magistrátu usedne nový primátor. Řada lidí bude muset za pohotovostí cestovat do jiného města, statisíce motoristů zase čeká výměna řidičských průkazů. To vše přinese rok 2023. Redakce iDNES.cz přináší přehledně seznam hlavních změn."
1,idnes.cz,Česko vstoupilo do roku 2023. Zákaz pyrotechniky v centru Prahy nezabral,https://www.idnes.cz/praha/zpravy/silvestr-novy-rok-hasici-policie-zachranna-sluzba-pripravy-vyjezdy-pyrotechnika.A221231_145404_domaci_pitt,2023-01-01,"Česká republika vstoupila do nového roku 2023. Oslavy se konaly na řadě míst, tradičně největší byly v centru Prahy, kde i přes zákaz došlo k odpalování pyrotechniky. Na Václavském náměstí před půlnocí muž kopl do hlavy policistu. Do druhé hodiny ranní zasahovali hasiči u padesátky požárů, záchranáři využili i specializované vozidlo Golem."
2,idnes.cz,"Hodnocení skončeného předsednictví: Češi uspěli, ale neumí se tím pochlubit",https://www.idnes.cz/zpravy/domaci/ceske-predsednictvi-v-rade-eu-hodnoceni.A221213_104017_domaci_knn,2023-01-01,"Spolu s koncem roku předala Česká republika pomyslné žezlo předsednictví v Radě EU Švédsku. Tím zakončila půl rok, během kterého zodpovídala za její fungování a stála tak „v čele Evropy“. Podle odborníků i politiků, které redakce iDNES.cz oslovila, země v této zkoušce obstála. Úspěchy jsou ale málo vidět a vláda je nedokázala dostatečně „prodat“."
3,idnes.cz,Kalousek má novou favoritku. Podpořil na Hrad Nerudovou místo Fischera,https://www.idnes.cz/volby/volba-prezidenta-2023-kalousek-podpora-danuse-nerudove.A230101_085308_prezidentske-volby-2023_kop,2023-01-01,"Necelé dva týdny zbývají do volby nové hlavy státu a do tábora přívrženců bývalé rektorky Mendelovy univerzity v Brně Danuše Nerudové veřejně přešel bývalý ministr financí a expředseda TOP 09 Miroslav Kalousek. Dosud přitom podporoval senátora Pavla Fischera, který ale podle Kalouska nemá šanci na postup do druhého kola volby."
4,idnes.cz,"Zeman udělil Medaili za hrdinství hasiči, který zahynul při požáru roubenky",https://www.idnes.cz/zpravy/domaci/zeman-oceneni-medaile-hasic-pozar-dvorak-in-memoriam.A230101_093737_domaci_bur,2023-01-01,Prezident Miloš Zeman udělil Medaili za hrdinství in memoriam Janu Dvořákovi. Dobrovolný hasič zahynul v listopadu při požáru roubenky v Novém Boru - Arnultovicích.
5,idnes.cz,Malostranský zápisník: Se Zemanem končí košilaté označení českých poslanců,https://www.idnes.cz/zpravy/domaci/malostransky-zapisnik-milos-zeman-vlada-poslanec.A221230_150203_domaci_javu,2023-01-01,"Kam se hrabe Babišova žvanírna, Zeman říkal Sněmovně mnohem hůř! Jak chce vláda zmodernizovat výuku oxymóronu v českých školách? Jak pravicová vláda dotuje stamiliony nezávislá média, aby psala správně? A těšíte se na Polčákův návrat? V Malostranském zápisníku píše komentátor MF DNES Miroslav Korecký."
6,idnes.cz,Prvním miminkem roku je Anička ze Šternberka. Minutu po ní se narodila Timea,https://www.idnes.cz/zpravy/domaci/prvni-miminko-dite-novorozenec-porodnice-2023-sternberk-nemocnice-anicka.A230101_120454_domaci_klf,2023-01-01,"Prvním dítětem narozeným v letošním roce v Česku je Anička. Přišla na svět v nemocnici ve Šternberku na Olomoucku osm minut po půlnoci. O minutu později se v Ústavu pro péči o matku a dítě v pražském Podolí narodila holčička Timea. Mezi prvními novorozenci v krajích převažují dívky, chlapečků je jen pět."
7,idnes.cz,Požárů na silvestra bylo o třetinu víc než loni. Hasiči zachraňovali i z komína,https://www.idnes.cz/zpravy/domaci/silvestr-zachranna-sluzba-zachranka-ohnostroj-petardy-zraneni-zasah-noc-novy-rok.A230101_073804_domaci_klf,2023-01-01,Hasiči v Česku během silvestrovské noci zasahovali u 286 požárů. Je to asi o třetinu víc než loni. Zásahy u požárů také tvořily tři čt

In [5]:
news[news["Abstract"].isna()]

,Web,Title,Url,Datetime,Abstract
280,"Pronajímatel má ze zákona právo na jednostranné zvýšení nájemného za pronájem bytu. Při zvyšování musí však být dodrženy zákonné podmínky, jakož i případná ujednání obsažená v nájemní smlouvě.",NaN,NaN,NaN,NaN


In [7]:
duplicate_rows = news[news.duplicated()]
print("Duplicate Rows:")
print(duplicate_rows)

Duplicate Rows:
Empty DataFrame
Columns: [Web, Title, Url, Datetime, Abstract]
Index: []


In [8]:
print('the number of articles before processing :',len(news))
news = news.dropna(subset=['Abstract'])
print('The number of articles after processing :',len(news))

the number of articles before processing : 683
The number of articles after processing : 682


In [10]:
news.reset_index(drop=True, inplace=True)

In [11]:
news

,Web,Title,Url,Datetime,Abstract
0,idnes.cz,PŘEHLEDNĚ: Co nového nás čeká v roce 2023. Za co více zaplatíme i dostaneme,https://www.idnes.cz/zpravy/domaci/cesko-zmeny-v-roce-2023-prehledne.A221230_105559_domaci_knn,2023-01-01,"Česko si zvolí nového prezidenta, na pražském magistrátu usedne nový primátor. Řada lidí bude muset za pohotovostí cestovat do jiného města, statisíce motoristů zase čeká výměna řidičských průkazů. To vše přinese rok 2023. Redakce iDNES.cz přináší přehledně seznam hlavních změn."
1,idnes.cz,Česko vstoupilo do roku 2023. Zákaz pyrotechniky v centru Prahy nezabral,https://www.idnes.cz/praha/zpravy/silvestr-novy-rok-hasici-policie-zachranna-sluzba-pripravy-vyjezdy-pyrotechnika.A221231_145404_domaci_pitt,2023-01-01,"Česká republika vstoupila do nového roku 2023. Oslavy se konaly na řadě míst, tradičně největší byly v centru Prahy, kde i přes zákaz došlo k odpalování pyrotechniky. Na Václavském náměstí před půlnocí muž kopl do hlavy policistu. Do druhé hodiny ranní zasahovali hasiči u padesátky požárů, záchranáři využili i specializované vozidlo Golem."
2,idnes.cz,"Hodnocení skončeného předsednictví: Češi uspěli, ale neumí se tím pochlubit",https://www.idnes.cz/zpravy/domaci/ceske-predsednictvi-v-rade-eu-hodnoceni.A221213_104017_domaci_knn,2023-01-01,"Spolu s koncem roku předala Česká republika pomyslné žezlo předsednictví v Radě EU Švédsku. Tím zakončila půl rok, během kterého zodpovídala za její fungování a stála tak „v čele Evropy“. Podle odborníků i politiků, které redakce iDNES.cz oslovila, země v této zkoušce obstála. Úspěchy jsou ale málo vidět a vláda je nedokázala dostatečně „prodat“."
3,idnes.cz,Kalousek má novou favoritku. Podpořil na Hrad Nerudovou místo Fischera,https://www.idnes.cz/volby/volba-prezidenta-2023-kalousek-podpora-danuse-nerudove.A230101_085308_prezidentske-volby-2023_kop,2023-01-01,"Necelé dva týdny zbývají do volby nové hlavy státu a do tábora přívrženců bývalé rektorky Mendelovy univerzity v Brně Danuše Nerudové veřejně přešel bývalý ministr financí a expředseda TOP 09 Miroslav Kalousek. Dosud přitom podporoval senátora Pavla Fischera, který ale podle Kalouska nemá šanci na postup do druhého kola volby."
4,idnes.cz,"Zeman udělil Medaili za hrdinství hasiči, který zahynul při požáru roubenky",https://www.idnes.cz/zpravy/domaci/zeman-oceneni-medaile-hasic-pozar-dvorak-in-memoriam.A230101_093737_domaci_bur,2023-01-01,Prezident Miloš Zeman udělil Medaili za hrdinství in memoriam Janu Dvořákovi. Dobrovolný hasič zahynul v listopadu při požáru roubenky v Novém Boru - Arnultovicích.
5,idnes.cz,Malostranský zápisník: Se Zemanem končí košilaté označení českých poslanců,https://www.idnes.cz/zpravy/domaci/malostransky-zapisnik-milos-zeman-vlada-poslanec.A221230_150203_domaci_javu,2023-01-01,"Kam se hrabe Babišova žvanírna, Zeman říkal Sněmovně mnohem hůř! Jak chce vláda zmodernizovat výuku oxymóronu v českých školách? Jak pravicová vláda dotuje stamiliony nezávislá média, aby psala správně? A těšíte se na Polčákův návrat? V Malostranském zápisníku píše komentátor MF DNES Miroslav Korecký."
6,idnes.cz,Prvním miminkem roku je Anička ze Šternberka. Minutu po ní se narodila Timea,https://www.idnes.cz/zpravy/domaci/prvni-miminko-dite-novorozenec-porodnice-2023-sternberk-nemocnice-anicka.A230101_120454_domaci_klf,2023-01-01,"Prvním dítětem narozeným v letošním roce v Česku je Anička. Přišla na svět v nemocnici ve Šternberku na Olomoucku osm minut po půlnoci. O minutu později se v Ústavu pro péči o matku a dítě v pražském Podolí narodila holčička Timea. Mezi prvními novorozenci v krajích převažují dívky, chlapečků je jen pět."
7,idnes.cz,Požárů na silvestra bylo o třetinu víc než loni. Hasiči zachraňovali i z komína,https://www.idnes.cz/zpravy/domaci/silvestr-zachranna-sluzba-zachranka-ohnostroj-petardy-zraneni-zasah-noc-novy-rok.A230101_073804_domaci_klf,2023-01-01,Hasiči v Česku během silvestrovské noci zasahovali u 286 požárů. Je to asi o třetinu víc než loni. Zásahy u požárů také tvořily tři čt

In [12]:
bert_uncased_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [13]:
class BertBaseDataset(Dataset):
    def __init__(self, tokenizer, df, text_col, max_len):
        self.tokenizer = tokenizer
        self.texts = df[text_col].values.tolist()
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        res = self.tokenizer(
            self.texts[idx],
            return_tensors="pt",
            padding=False,
            truncation=True,
            # return_special_tokens_mask=True,
            max_length=self.max_len,
        )
        res = {k:v[0] for k,v in res.items()}
        return res
    
def dynamic_pad_data_loader(tokenizer, dataset, batch_size, shuffle):
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=data_collator
    )

In [14]:
backbone = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(backbone)

ds = BertBaseDataset(tokenizer, news, "Abstract", None)
dl = dynamic_pad_data_loader(tokenizer, ds, batch_size=32, shuffle=False)

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(backbone)
model.eval()

preds = []

with torch.no_grad(), tqdm(total=len(dl)) as pbar:
    for batch in dl:
        output = model(**batch)
        preds.append(output["logits"])
        pbar.update(1)

# Concatenate the logits from all batches
preds = torch.cat(preds, dim=0)

print("Shape of all logits:", preds.shape)

100%|██████████| 22/22 [02:47<00:00,  7.62s/it]

Shape of all logits: torch.Size([682, 5])


In [16]:
preds = preds.softmax(1).numpy()
preds_df = pd.DataFrame(preds, columns=[f"{i+1} stars" for i in range(5)])
preds_df["predicted stars"] = preds.argmax(axis=1) + 1
preds_df.insert(0, "Title", news['Title'])
preds_df.insert(0, "Abstract", news["Abstract"])
preds_df.insert(0, "Datetime", news["Datetime"])
preds_df.insert(0, 'Web', news['Web'])
preds_df.head()

,Web,Datetime,Abstract,Title,1 stars,2 stars,3 stars,4 stars,5 stars,predicted stars
0,idnes.cz,2023-01-01,"Česko si zvolí nového prezidenta, na pražském magistrátu usedne nový primátor. Řada lidí bude muset za pohotovostí cestovat do jiného města, statisíce motoristů zase čeká výměna řidičských průkazů. To vše přinese rok 2023. Redakce iDNES.cz přináší přehledně seznam hlavních změn.",PŘEHLEDNĚ: Co nového nás čeká v roce 2023. Za co více zaplatíme i dostaneme,0.088154,0.072447,0.159511,0.369156,0.310732,4
1,idnes.cz,2023-01-01,"Česká republika vstoupila do nového roku 2023. Oslavy se konaly na řadě míst, tradičně největší byly v centru Prahy, kde i přes zákaz došlo k odpalování pyrotechniky. Na Václavském náměstí před půlnocí muž kopl do hlavy policistu. Do druhé hodiny ranní zasahovali hasiči u padesátky požárů, záchranáři využili i specializované vozidlo Golem.",Česko vstoupilo do roku 2023. Zákaz pyrotechniky v centru Prahy nezabral,0.213713,0.120329,0.124306,0.232894,0.308758,5
2,idnes.cz,2023-01-01,"Spolu s koncem roku předala Česká republika pomyslné žezlo předsednictví v Radě EU Švédsku. Tím zakončila půl rok, během kterého zodpovídala za její fungování a stála tak „v čele Evropy“. Podle odborníků i politiků, které redakce iDNES.cz oslovila, země v této zkoušce obstála. Úspěchy jsou ale málo vidět a vláda je nedokázala dostatečně „prodat“.","Hodnocení skončeného předsednictví: Češi uspěli, ale neumí se tím pochlubit",0.317008,0.359434,0.207930,0.082485,0.033143,2
3,idnes.cz,2023-01-01,"Necelé dva týdny zbývají do volby nové hlavy státu a do tábora přívrženců bývalé rektorky Mendelovy univerzity v Brně Danuše Nerudové veřejně přešel bývalý ministr financí a expředseda TOP 09 Miroslav Kalousek. Dosud přitom podporoval senátora Pavla Fischera, který ale podle Kalouska nemá šanci na postup do druhého kola volby.",Kalousek má novou favoritku. Podpořil na Hrad Nerudovou místo Fischera,0.448963,0.300542,0.145913,0.072200,0.032383,1
4,idnes.cz,2023-01-01,Prezident Miloš Zeman udělil Medaili za hrdinství in memoriam Janu Dvořákovi. Dobrovolný hasič zahynul v listopadu při požáru roubenky v Novém Boru - Arnultovicích.,"Zeman udělil Medaili za hrdinství hasiči, který zahynul při požáru roubenky",0.205436,0.065649,0.127099,0.260190,0.341625,5


In [17]:
len(preds_df)

682

In [18]:
preds_df

,Web,Datetime,Abstract,Title,1 stars,2 stars,3 stars,4 stars,5 stars,predicted stars
0,idnes.cz,2023-01-01,"Česko si zvolí nového prezidenta, na pražském magistrátu usedne nový primátor. Řada lidí bude muset za pohotovostí cestovat do jiného města, statisíce motoristů zase čeká výměna řidičských průkazů. To vše přinese rok 2023. Redakce iDNES.cz přináší přehledně seznam hlavních změn.",PŘEHLEDNĚ: Co nového nás čeká v roce 2023. Za co více zaplatíme i dostaneme,0.088154,0.072447,0.159511,0.369156,0.310732,4
1,idnes.cz,2023-01-01,"Česká republika vstoupila do nového roku 2023. Oslavy se konaly na řadě míst, tradičně největší byly v centru Prahy, kde i přes zákaz došlo k odpalování pyrotechniky. Na Václavském náměstí před půlnocí muž kopl do hlavy policistu. Do druhé hodiny ranní zasahovali hasiči u padesátky požárů, záchranáři využili i specializované vozidlo Golem.",Česko vstoupilo do roku 2023. Zákaz pyrotechniky v centru Prahy nezabral,0.213713,0.120329,0.124306,0.232894,0.308758,5
2,idnes.cz,2023-01-01,"Spolu s koncem roku předala Česká republika pomyslné žezlo předsednictví v Radě EU Švédsku. Tím zakončila půl rok, během kterého zodpovídala za její fungování a stála tak „v čele Evropy“. Podle odborníků i politiků, které redakce iDNES.cz oslovila, země v této zkoušce obstála. Úspěchy jsou ale málo vidět a vláda je nedokázala dostatečně „prodat“.","Hodnocení skončeného předsednictví: Češi uspěli, ale neumí se tím pochlubit",0.317008,0.359434,0.207930,0.082485,0.033143,2
3,idnes.cz,2023-01-01,"Necelé dva týdny zbývají do volby nové hlavy státu a do tábora přívrženců bývalé rektorky Mendelovy univerzity v Brně Danuše Nerudové veřejně přešel bývalý ministr financí a expředseda TOP 09 Miroslav Kalousek. Dosud přitom podporoval senátora Pavla Fischera, který ale podle Kalouska nemá šanci na postup do druhého kola volby.",Kalousek má novou favoritku. Podpořil na Hrad Nerudovou místo Fischera,0.448963,0.300542,0.145913,0.072200,0.032383,1
4,idnes.cz,2023-01-01,Prezident Miloš Zeman udělil Medaili za hrdinství in memoriam Janu Dvořákovi. Dobrovolný hasič zahynul v listopadu při požáru roubenky v Novém Boru - Arnultovicích.,"Zeman udělil Medaili za hrdinství hasiči, který zahynul při požáru roubenky",0.205436,0.065649,0.127099,0.260190,0.341625,5
5,idnes.cz,2023-01-01,"Kam se hrabe Babišova žvanírna, Zeman říkal Sněmovně mnohem hůř! Jak chce vláda zmodernizovat výuku oxymóronu v českých školách? Jak pravicová vláda dotuje stamiliony nezávislá média, aby psala správně? A těšíte se na Polčákův návrat? V Malostranském zápisníku píše komentátor MF DNES Miroslav Korecký.",Malostranský zápisník: Se Zemanem končí košilaté označení českých poslanců,0.561323,0.246295,0.107634,0.051510,0.033238,1
6,idnes.cz,2023-01-01,"Prvním dítětem narozeným v letošním roce v Česku je Anička. Přišla na svět v nemocnici ve Šternberku na Olomoucku osm minut po půlnoci. O minutu později se v Ústavu pro péči o matku a dítě v pražském Podolí narodila holčička Timea. Mezi prvními novorozenci v krajích převažují dívky, chlapečků je jen pět.",Prvním miminkem roku je Anička ze Šternberka. Minutu po ní se narodila Timea,0.176296,0.201573,0.194400,0.244075,0.183655,4
7,idnes.cz,2023-01-01,Hasiči v Česku během silvestrovské noci zasahovali u 286 požárů. Je to asi o třetinu víc než loni. Zásahy u požárů také tvořily tři čtvrtiny všech výjezdů hasičů. Záchranáři v krajích pak kromě zranění petardami ošetřovali i oči poraněné při otevírání sektu a opařené dítě.,Požárů na silvestra bylo o třetinu víc než loni. Hasiči zachraňovali i z komína,0.785202,0.121507,0.043305,0.019971,0.030014,1
8,idnes.cz,2023-01-01,"Loňský rok byl na českých silnicích zřejmě druhý nejméně tragický za víc než 60 let. Obětí nehod bylo podle předběžných údajů policie 468, což bylo o dvě nehody méně než o rok dřív.",Obětí nehod ubylo. Loňský rok byl na silnicích jeden z méně tragických,0.645269,0.250444,0.067852,0.018977,0.017458,1
9,idnes.cz,2023-01-01,"S novoročním projevem vystoupil předseda Senátu 

In [19]:
csv_path = "/kaggle/working/preds_abstract_web.csv"

# Save the DataFrame to a CSV file
preds_df.to_csv(csv_path, index=False)

In [21]:
c = news[['Web']].value_counts()

index = []
for i in c.index:
    index.append(np.array(i))
index = np.array(index)

df = pd.DataFrame(columns=['Category','Values'])
df['Web'] = index[:,0]
df['Values'] = c.values

px.bar(data_frame=df,x='Web',y='Values')

In [22]:
preds_df.groupby(['predicted stars', 'Web']).size().unstack(fill_value=0)

Web,ac24.cz,aktualne.cz,czechfreepress.cz,idnes.cz,lidovky.cz
predicted stars,,,,,
1,49,48,0,79,155
2,9,24,2,21,61
3,5,4,0,6,37
4,14,16,0,27,60
5,6,8,2,8,41


In [23]:
result_df = preds_df.groupby(['predicted stars', 'Web']).size().unstack(fill_value=0)

# Normalize each element by dividing it by the sum of its column
result_df = result_df.div(result_df.sum(axis=0), axis=1) * 100

# Print the results
for stars in result_df.index:
    print(f'\nTop webs for {stars} stars (percentage of total count):')
    print(result_df.loc[stars].sort_values(ascending=False))


Top webs for 1 stars (percentage of total count):
Web
ac24.cz              59.036145
idnes.cz             56.028369
aktualne.cz          48.000000
lidovky.cz           43.785311
czechfreepress.cz     0.000000
Name: 1, dtype: float64

Top webs for 2 stars (percentage of total count):
Web
czechfreepress.cz    50.000000
aktualne.cz          24.000000
lidovky.cz           17.231638
idnes.cz             14.893617
ac24.cz              10.843373
Name: 2, dtype: float64

Top webs for 3 stars (percentage of total count):
Web
lidovky.cz           10.451977
ac24.cz               6.024096
idnes.cz              4.255319
aktualne.cz           4.000000
czechfreepress.cz     0.000000
Name: 3, dtype: float64

Top webs for 4 stars (percentage of total count):
Web
idnes.cz             19.148936
lidovky.cz           16.949153
ac24.cz              16.867470
aktualne.cz          16.000000
czechfreepress.cz     0.000000
Name: 4, dtype: float64

Top webs for 5 stars (percentage of total count):
Web
czechfree